In [ ]:
import os, gc
os.chdir( '/Users/user/ownCloud/' )

In [ ]:
import warnings ; warnings.simplefilter( "ignore" )
import numpy as np, pandas as pd, flex as fl
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
mnist_npz = '/Users/user/study_notes/year_14_15/spring_2015/machine_learning/data/mldata/mnist_scikit.npz'
assert( os.path.exists( mnist_npz ) )
with np.load( mnist_npz, 'r' ) as npz :
    mnist_labels, mnist_data = np.asarray( npz[ 'labels' ], np.int ), npz[ 'data' ] / 255.0

In [ ]:
from sklearn import *

In [ ]:
split_ = cross_validation.train_test_split( mnist_data, mnist_labels, test_size = 0.75 )
X_train, X_test, y_train, y_test = split_

In [ ]:
print X_train.shape
print X_test.shape

## Simple average pattern classifier

Generate averaged images

In [ ]:
mnist_labels_ = np.unique( mnist_labels )

train_digits_ = { label_ : X_train[ np.flatnonzero( y_train == label_ ) ]
                           for label_ in mnist_labels_ }

avg_patterns_ = { label_ : samples_.mean( axis = 0 )[ np.newaxis ]
                  for label_, samples_ in train_digits_.iteritems() }

## Construct an aligned pair of arrays
average_digits = np.concatenate( [ array_ for array_ in avg_patterns_.itervalues() ], axis = 0 )
average_labels = np.array( [ label_ for label_ in avg_patterns_.iterkeys() ], dtype = np.int )

## Sort them
order_ = np.argsort( average_labels )[::-1]
average_labels = average_labels[ order_ ] ; average_digits = average_digits[ order_ ] ; 

In [ ]:
## Plot the patterns
axis = plt.figure( figsize = ( 16, 9 ) ).add_subplot( 111 )
axis.set_title( u"Averaged digits on train" )
fl.plot( axis, average_digits, shape = ( 28, -1 ), cmap = plt.cm.hot, interpolation = "nearest" )
plt.show( )

A simple MAP calssifier based on mean patterns

In [ ]:
## Compute the squared error ...
yp_ = - 2 * np.tensordot( X_test, average_digits, axes = [ -1, -1 ] ) \
      + average_digits.dot( average_digits.T )[:1]
## ... and based on it find the closest label.
pred_labels_ = average_labels[ yp_.argmin(axis = 1) ]

Construct the confusion matrix

In [ ]:
import scipy.sparse as sp
tbl_ = sp.coo_matrix( ( np.ones_like( pred_labels_ ), ( pred_labels_, y_test ) ) ).todense( )
tbl = pd.DataFrame( tbl_, index = average_labels, columns = average_labels )
tbl.index.name = "Predicted" ; tbl.columns.name = "Actual" ;

tbl

Compute the accuracy

In [ ]:
print "Achieved accuracy is %.3f%%" % ( np.mean( y_test == pred_labels_ ) * 100.0, )

## Random Forest WTF!

In [ ]:
rfc_ = ensemble.RandomForestClassifier( n_estimators = 50, n_jobs = -1, random_state = None ).fit( X_train, y_train )

In [ ]:
pred_labels_ = rfc_.classes_[ rfc_.predict_proba( X_test ).argmax( axis = 1 ) ]

In [ ]:
print "Achieved accuracy is %.3f%%" % ( np.mean( y_test == pred_labels_ ) * 100.0, )

In [ ]:
import scipy.sparse as sp
tbl_ = sp.coo_matrix( ( np.ones_like( pred_labels_ ), ( pred_labels_, y_test ) ) ).todense( )
tbl = pd.DataFrame( tbl_, index = average_labels, columns = average_labels )
tbl.index.name = "Predicted" ; tbl.columns.name = "Actual" ;

tbl

In [ ]:
## Plot the patterns
axis = plt.figure( figsize = ( 16, 9 ) ).add_subplot( 111 )
axis.set_title( u"Random Forest feature importances" )
fl.plot( axis, rfc_.feature_importances_[ np.newaxis ], n_row = 1, n_col = 1,
         shape = ( 28, -1 ), cmap = plt.cm.hot, interpolation = "nearest" )
plt.show( )

Draw and plot some random subset of train dataset.

In [ ]:
index_perm_ = np.random.permutation( X_train.shape[ 0 ] )
X = mnist_data[ index_perm_[ : 400 ] ]

In [ ]:
axis = plt.figure( figsize = ( 16, 9 ) ).add_subplot( 111 )
axis.set_title( u"200 random MNIST digits." )
fl.plot( axis, X, n = 400, shape = ( 28, -1 ), cmap = plt.cm.hot, interpolation = "nearest" )
plt.show( )

<hr/>

In [ ]:
import lasagne, theano.tensor as T

Implement as simple neural network with lasagne

In [ ]:
input_var = T.tensor4( 'inputs' )
target_var = T.ivector( 'targets' ) 

In [ ]:
## The input layer
network = lasagne.layers.InputLayer( shape = ( None, 1, 28, 28 ),
                                     input_var = input_var )

## The 2D 5x5 conv layer with ReLU and 2x2 max-pooling
## (28-7+1) // 1
network = lasagne.layers.Conv2DLayer( network, num_filters = 32, filter_size = ( 5, 5 ),
                                      nonlinearity = lasagne.nonlinearities.rectify,
                                      border_mode = "valid", W = lasagne.init.GlorotUniform( ) )
network = lasagne.layers.MaxPool2DLayer( network, pool_size = ( 2, 2 ) )

## The 2D 3x3 conv layer with ReLU and 2x2 max-pooling
network = lasagne.layers.Conv2DLayer( lasagne.layers.dropout( network, p = .2 ),
                                      num_filters = 64, filter_size = ( 3, 3 ),
                                      nonlinearity = lasagne.nonlinearities.rectify,
                                      border_mode = "valid", W = lasagne.init.GlorotUniform( ) )
network = lasagne.layers.MaxPool2DLayer( network, pool_size = ( 2, 2 ) )

## FC layer with dropout
network = lasagne.layers.DenseLayer( lasagne.layers.dropout( network, p = .5 ),
                                     num_units = 256, nonlinearity = lasagne.nonlinearities.rectify )

network = lasagne.layers.DenseLayer( lasagne.layers.dropout( network, p = .5 ),
                                     num_units = 10, nonlinearity = lasagne.nonlinearities.softmax )


In [ ]:
prediction = lasagne.layers.get_output( network )
loss = lasagne.objectives.categorical_crossentropy( prediction, target_var ).mean( )

In [ ]:
params = lasagne.layers.get_all_params( network, trainable = True )
updates = lasagne.updates.nesterov_momentum( loss, params, learning_rate = 0.01, momentum = 0.9 )

In [ ]:
test_prediction = lasagne.layers.get_output( network, deterministic = True )
test_loss = lasagne.objectives.categorical_crossentropy( test_prediction, target_var ).mean( )

test_acc = T.mean( T.eq( T.argmax( test_prediction, axis = 1 ), target_var ), dtype = theano.config.floatX )

In [ ]:
train_fn = theano.function([input_var, target_var], loss, updates=updates)
val_fn = theano.function([input_var, target_var], [test_loss, test_acc])